<center><h1><b>Real or Not? NLP with Disaster Tweet</b></h1></center>
<center>DSI206 Multimedia Representation Management</center>

### รายชื่อสมาชิก
> 1. นางสาวอโรชา หมอสินธุ์ 6209656104
> 2. นางสาวศศิมา นิ่มมา 6209656013
> 3. นางสาวรวงข้าว แสงทวีปทวีกิจ 6209656146

# Introduction
#### ในปัจจุบัน twitter เป็นที่นิยมมากในการใช้ติดต่อสื่อสารที่สำคัญ เพราะมีความรวดเร็ว สามารถเฝ้าสังเกตสถานการณ์ฉุกเฉินได้แบบเรียลไทม์ จึงทำให้ผู้คนส่วนใหญ่เลือกที่จะใช้ twitter ในการการกระจายข่าวสารต่างๆ ดังนั้นจึงมีความสนใจการทำนายข้อความจาก twitter ว่าเป็นข้อความที่เกี่ยวข้องกับภัยพิบัติหรือไม่ โดยจะสร้างโมเดลแมชชีนเลิร์นนิ่งที่คาดการณ์ว่าทวีตใดเกี่ยวกับภัยพิบัติจริงและไม่จริง

# Data Set
### 1. train คือ ข้อมูลที่ใช้การการฝึก Machine Learning
> * id - เลข id ผู้ใช้งาน twitter
> * text - ข้อความที่ผู้ใช้งาน twitter ทวีต
> * location - ตำแหน่งของผู้ใช้งานทวีต ณ เวลาที่ทวีต
> * keyword - คำที่ใช้ในการทำนาย
> * target - ผลการทำนายว่าเกี่ยวข้องกับภัยพิบัติหรือไม่

### 2. test คือ ข้อมูลที่รวบรวมข้อมูลที่ใช้ในการทดสอบ Machine Learning
> * id - เลข id ผู้ใช้งาน twitter
> * text - ข้อความที่ผู้ใช้งาน twitter ทวีต
> * location - ตำแหน่งของผู้ใช้งานทวีต ณ เวลาที่ทวีต
> * keyword - คำที่ใช้ในการทำนาย

### 3. sample_submission.csv คือ ไฟล์ที่เก็บข้อมูลในรูปแบบที่ผ่านการวิเคราะห์แล้ว

### ทำการ import module ดังนี้
> * ประกาศขอใช้ numpy สำหรับช่วยในการจัดการค่าต่างๆ ที่เกี่ยวการคำนวณ 
> * ประกาศขอใช้ pandas สำหรับจัดการข้อมูลและแสดงผลข้อมูลแบบกราฟิก
> * ประกาศขอใช้ os สำหรับช่วยในการเช็คข้อมูลใน dataset ว่ามีอะไรบ้าง

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')
train.head()

# Cleaning
จากนั้นดึงไฟล์เก็บเข้าในตัวแปร แล้วทำการ cleaning data

> * ขอใช้ Regular Expression ในการดำเนินการกับข้อความในรูปแบบ string โดยการ import re และ import string

> * สร้างฟังก์ชัน contraction ในการเปลี่ยนแปลงตัวอักขระหรือคำบางคำที่เป็นตัวย่อให้กลายเป็นคำรูปแบบเต็ม

> * ใช้ re.sub() ในการเขียนทับข้อความตรงส่วนนั้นๆ

> * หลังจากสร้างฟังก์ชันแล้วจึงทำการนำมาใช้กับตัวแปร train และ test โดยการใช้ lambda เข้ามาช่วย เพื่อลดรูปในการสร้างฟังก์ชัน


In [ ]:
import re
import string
def contraction(text):
    # specific
    text = re.sub(r"won\'t", "will not", text)
    # general
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text
    

train['text'] = train['text'].apply(lambda x : contraction(x))
test['text'] = test['text'].apply(lambda x : contraction(x))

train['text'].head()

> * สร้างฟังก์ชัน urls ในการช่วยในการตัด link

> * ใช้ re.sub() ในการลบข้อมูลตรงส่วนนั้นออก

> * หลังจากสร้างฟังก์ชันแล้วจึงทำการนำมาใช้กับตัวแปร train และ test โดยการใช้ lambda เข้ามาช่วย

In [ ]:
def urls(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return text

train['text'] = train['text'].apply(lambda x : urls(x))
test['text'] = test['text'].apply(lambda x : urls(x))

train['text'].head()


> * สร้างฟังก์ชัน spacial ในการช่วยในการตัดข้อความบางส่วนที่ไม่จำเป็นออกไป

> * ใช้ re.sub() ในการทำตัวอักษรให้เป็นตัวพิมพ์เล็กทั้งหมด รวมถึงลบเครื่องหมายต่างๆที่ไม่ต้องการ รวมไปถึงการขึ้นบรรทัดใหม่ เครื่องหมายวรรคตอน คำที่มีตัวเลขคั่น

> * หลังจากสร้างฟังก์ชันแล้วจึงทำการนำมาใช้กับตัวแปร train และ test โดยการใช้ lambda เข้ามาช่วย

In [ ]:
def spacial(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

train['text'] = train['text'].apply(lambda x : spacial(x))
test['text'] = test['text'].apply(lambda x : spacial(x))

train['text'].head()

> * สร้างฟังก์ชัน emoji ในการลบ emoji ออกไป

> * ใช้ re.compile() ในการรวบรวมรูปแบบที่เกี่ยวข้องกับ emoji โดยการแทน emoji เป็นรหัสในทุกๆหมวดหมู่ และลบโดยใช้ไลบรารี่ re

> * หลังจากสร้างฟังก์ชันแล้วจึงทำการนำมาใช้กับตัวแปร train และ test โดยการใช้ lambda เข้ามาช่วย

In [ ]:
def emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
train['text'] = train['text'].apply(lambda x : emoji(x))
test['text'] = test['text'].apply(lambda x : emoji(x))

train['text'].head()

In [ ]:
train.head()

In [ ]:
test.head()

# Analyzing 
ทางกลุ่มได้ทำการ import nltk เพื่อนำมาใช้กับ stopwords และ import CountVectorizer จาก sklearn เพื่อแปลงชุดข้อความให้กลายเป็นเลข โดยจะแยกตามตำแหน่งของคำ

> * token_pattern : รูปแบบของ token ที่จะทำการวิเคราะห์
> * ngram_range : กำหนดขอบเขตการเลือกคำที่ติดกัน n คำ และให้ค่าความน่าจะเป็นของชุดคำที่เกิดขึ้นร่วมกัน
> * stop_words : ตัวแปรในการตัดคำที่พบบ่อยในประโยคแต่ไม่สื่อถึงอะไร โดยใช้ตัวแปร stopwords ที่เก็บ list ของคำภาษาอังกฤษที่พบบ่อย

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

stopwords = stopwords.words('english')

count_vector = CountVectorizer(token_pattern=r'\w{1,}', ngram_range=(1, 2), stop_words = stopwords)

ขอใช้คำสั่ง train_test_split จากไลบรารี่ sklearn

* Train ใช้สำหรับป้อนให้โมเดลใช้เทรน
* Test ใช้สำหรับทดสอบหา Metrics หลังจากเทรนเสร็จว่า โมเดลจะทำงานได้ดีแค่ไหน

สร้างตัวแปรอิสระ X เป็นฟีทเจอร์หรือคุณสมบัติ

สร้างตัวแปรตาม y เป็นเลเบล (label) ซึ่งเป็นคำตอบที่ต้องการ 

สร้างตัวแปร X_train, X_test , y_train และ y_test ด้วยฟังก์ชั่น train_test_split() กำหนด test_size 10% (0.1)

In [ ]:
from sklearn.model_selection import train_test_split
X = train.text
y = train.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

ขอใช้ LogisticRegression จากไลบรารี่ sklearn

ขอใช้ Pipeline จากไลบรารี่ sklearn

clf : ตัวแปรที่เก็บฟังก์ชัน LogisticRegression

pipe : ซึ่งการทำงานของมันคือทำงานเป็นลำดับเหมือนท่อ โดยเราสามารถใส่ฟังก์ชั่นเข้าไปท่อนี้ดังโค้ดข้างล่าง คือการนำเอาฟังก์ชั่น CountVectorizer ไว้จัดการ จากนั้นนำค่าไป scale แล้วก็เข้าโมเดลของเราคือ LogisticRegression นั้นเอง

pipe.fit() แต่ละตัวภายในไปป์ไลน์จะติดตั้งกับเอาต์พุตของก่อนหน้า


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = LogisticRegression()
pipe = Pipeline([
    ('count_vector', CountVectorizer()),  
    ('clf', LogisticRegression()) 
])
pipe.fit(X_train,y_train)

จากนั้นทางกลุ่มจะมาดูว่าความถูกต้องในการแยกแยะข้อมูลแต่ละรายการในเซ็ตข้อมูลนั้น แม่นยำเพียงไร 

ขอใช้ metrics จากไลบรารี่ sklearn เพื่อทำการทำนายผลความถูกต้องแม่นยำ 

ทางกลุ่มจะทำการทำนาย predict กับ y_test ซึ่งจะดูว่าผลการทำนายกับผลลัพท์ที่ถูกต้องจากเลเบลว่ามีมากน้อยเพียงไร และทำการแจกแจงคะแนนเพื่อให้สามารถเข้าใจได้ง่ายขึ้น

In [ ]:
from sklearn import metrics
predicted = pipe.predict(X_test)

In [ ]:
print("accuracy :",  metrics.accuracy_score(predicted, y_test))

# Submission

ทำการเก็บ model ที่ได้มา predict กับ test.text ลงในไฟล์ submission ในคอลัมม์ target จากนั้นอัพโหลดเข้าไฟล์ submission.csv เพื่อทำการส่งผลการทำนาย

In [ ]:
submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
submission["target"] = pipe.predict(test.text)
submission.to_csv("submission.csv", index=False)

In [ ]:
submission.head(10)

# Reference
[https://medium.com/@minatonamikazerak/python-%E0%B8%A5%E0%B8%AD%E0%B8%87%E0%B9%80%E0%B8%A5%E0%B9%88%E0%B8%99-regular-expression-aeffdb07c794](http://)

[https://python3.wannaphong.com/2014/10/python-regular-expressions.html](http://)

[https://qastack.in.th/programming/33091376/python-what-is-exactly-sklearn-pipeline-pipeline](http://)

[https://www.kaggle.com/iavinas/nlp-spacy-basics](http://)

[https://www.kaggle.com/patipanrattanawin/disaster-tweet-text-classification-simplest-way](http://)